In [1]:
import requests
from tqdm.auto import tqdm
import pandas as pd
import ssl
import nltk
import time
import math
import glob
import pymorphy2
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Раздел 1: Разработка и анализ

## Сбор вакансий

In [2]:
# определим работодателей для сбора вакансий
employers = {'name': ['sber', 'sbertech', 'rosatom', 'gazprom_neft'], 'id': [3529, 906557, 577743, 39305]}

num_per_page = 100
all_vacancy_ids = []

# соберем идентификаторы вакансий по каждому работодателю
for employer in employers['id']:
    url = f'https://api.hh.ru/vacancies?employer_id={employer}&per_page={num_per_page}'
    res = requests.get(url)
    number_of_pages = res.json().get('pages')
    
    for i in tqdm(range(number_of_pages)):
        url = f'https://api.hh.ru/vacancies?employer_id={employer}&page={i}&per_page={num_per_page}'
        res = requests.get(url)
        vacancies = res.json()
        vacancy_ids = [el.get('id') for el in vacancies.get('items')]
        all_vacancy_ids.extend(vacancy_ids)
        

100%|███████████████████████████████████████████| 20/20 [00:07<00:00,  2.61it/s]


In [3]:
# число вакансий
print(len(all_vacancy_ids))

4170


In [4]:
# проверим состав получаемых полей
url = f'https://api.hh.ru/vacancies/54162408'

r = requests.get(url)

# нормализуем для удобства
j = pd.json_normalize(r.json(), max_level=1)

print(j.columns)

Index(['id', 'premium', 'relations', 'name', 'insider_interview',
       'response_letter_required', 'address', 'allow_messages', 'contacts',
       'description', 'branded_description', 'vacancy_constructor_template',
       'key_skills', 'accept_handicapped', 'accept_kids', 'archived',
       'response_url', 'specializations', 'professional_roles', 'code',
       'hidden', 'quick_responses_allowed', 'driver_license_types',
       'accept_incomplete_resumes', 'published_at', 'created_at',
       'negotiations_url', 'suitable_resumes_url', 'apply_alternate_url',
       'has_test', 'test', 'alternate_url', 'working_days',
       'working_time_intervals', 'working_time_modes', 'accept_temporary',
       'billing_type.id', 'billing_type.name', 'area.id', 'area.name',
       'area.url', 'salary.from', 'salary.to', 'salary.currency',
       'salary.gross', 'type.id', 'type.name', 'experience.id',
       'experience.name', 'schedule.id', 'schedule.name', 'employment.id',
       'employment.n

In [5]:
# соберем данные по вакансиям

# рассчитаем число шагов для сбора (для минимизации шансов отключения сессии)
step = 200
number_of_steps = math.ceil(len(all_vacancy_ids)/step)

for stp in range(number_of_steps):
    all_vacancy_descriptions=pd.DataFrame(columns=['id', 'name', 'description', 'employer.name'])
    loop_ids = all_vacancy_ids[stp*step:(stp+1)*step]
    
    for i in tqdm(loop_ids):
        
        url = f'https://api.hh.ru/vacancies/{i}'
        res = requests.get(url)
        # нормализация json
        vacancies = pd.json_normalize(res.json(), max_level=1).filter(['id', 'name', 'description', 'employer.name'])
        
        all_vacancy_descriptions = pd.concat([all_vacancy_descriptions, vacancies], join="outer")
    
    # сохранение в csv
    all_vacancy_descriptions.to_csv(f'descriptions_{stp}.csv')
    # ожидание, чтобы минимизировать шансы на завершение сессии со стороны API hh.ru
    time.sleep(10)
    

100%|█████████████████████████████████████████| 170/170 [00:23<00:00,  7.21it/s]


## Разработка алгоритма ранжирования

In [26]:
# загрузим сохраненные данные
df = pd.DataFrame()

for f in glob.glob('descriptions_*.csv'):
    df = pd.concat([df, pd.read_csv(f, index_col='id')], join="outer")
    
df = df.drop(columns='Unnamed: 0')

In [27]:
# проверим есть ли вакансии без описания
df[df.description.isna()]

,name,description,employer.name
id,,,


In [28]:
# скачаем стопслова
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')

stopwords_rus = stopwords.words('russian')

# дополним лишними элементами (html, css)
stopwords_rus.extend(['<strong>', '</strong>', 'strong', '<ul>', '</ul>', 'ul', '<li>', '</li>', 'li', '<p>', '</p>', 'p', 'n', 'u'])


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ilyanaboyshchikov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
# загрузим описание опыта
with open('Описание опыта.txt', encoding='utf-8') as f:
    experience = pd.Series(f.read())

In [42]:
# объединим датафрейм с описанями вакансий и строку с опытом кандидата
description_with_experience = pd.concat([df.description, experience], join="outer")
description_with_experience = description_with_experience.reset_index(drop=True)

description_with_experience.iloc[[0, -1]].replace('\n', ' ').replace('\u2028',' ')

0       <strong>Обязанности:</strong> <ul> <li>Обеспечивать предоставление Компанией услуг на установленном уровне SLA с четко контролируемыми метриками и отчетностью по услугам для каждого сервиса,</li> <li>Осуществлять методологическую поддержку специалистов производственных подразделений по вопросам оказания сервиса на основании установленных правил и принятых НМД с целью организации компетентных сервисных команд, для обеспечения эффективной, корректной ИТ-поддержки, отвечающей на установленные бизнес-задачи.</li> <li>Вести проактивную работу, направленную на прогнозирование влияния инцидентов на все бизнес-системы в зоне ответственности Управления, с целью предотвращения остановок в работе систем и предотвращения новых инцидентов.</li> <li>Обеспечивать план-фактный анализ работы всех бизнес-систем в зоне ответственности Управления.</li> <li>Контролировать разработку планов аварийного восстановления Систем, с целью предотвращения простоя бизнес-критичных систем в случае возникновени

In [43]:
# функция лемматизации
def lemmatize(row):
    morph = pymorphy2.MorphAnalyzer()
    t = []
    for word in row.split():
        if word in ['strong', 'ul', 'li']:
            t.append('')
        elif len(word)<=3:
            continue
        else:
            p = morph.parse(word)[0]
            t.append(p.normal_form)
    return " ".join(t)

# применим к датасету
description_with_experience.apply(lemmatize)

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [44]:
# настроим TF-IDF
text_transformer = TfidfVectorizer(stop_words=stopwords_rus, ngram_range=(1,1), lowercase=True, max_features=10000)
text = text_transformer.fit_transform(description_with_experience)
text

<3872x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 542230 stored elements in Compressed Sparse Row format>

In [45]:
# посмотрим на полученный словарь обработанных слов
list(text_transformer.vocabulary_.keys())[:10]

['обязанности',
 'обеспечивать',
 'предоставление',
 'компанией',
 'услуг',
 'установленном',
 'уровне',
 'sla',
 'четко',
 'метриками']

In [46]:
pd.set_option('display.max_colwidth', None)

# рассчитаем матрицу косинусных расстояний
cos_similarity_matrix = cosine_similarity(text)

# возьмем только столбец с расстояниями для опыта кандидата
cos_similarity_series = pd.Series(cos_similarity_matrix[-1])

# отбросим последнюю строку с расстоянием = 1 
cos_similarity_series = cos_similarity_series[:-1]

df = df[~df.index.duplicated()]
cos_similarity_series.index = df.index

# добавим полученные расстояния обратно в датасет
df['cos_similarity'] = cos_similarity_series[:-1]

# отсортируем по убыванию косинусного расстояния
df = df.sort_values(by='cos_similarity', ascending=False)

# отобразим 10 первых вакансий
df[['name', 'employer.name', 'cos_similarity']].head(10)

,name,employer.name,cos_similarity
id,,,
50854353,Ведущий разработчик Qlik,Газпром нефть,0.189731
55424240,Аналитик,Газпром нефть,0.176790
55850116,BI Главный специалист,Газпром нефть,0.171840
55247084,Аналитик данных,Сбербанк,0.166036
55846460,Разработчик SQL (PostgreSQL),Сбербанк,0.164689
55105469,BI аналитик / Аналитик данных,Сбербанк,0.163657
55858523,Ведущий инженер по разработке QlikView \ Qlik Sense,Сбербанк,0.162352
66118431,Аналитик продуктов,Газпром нефть,0.159912
66134564,Главный специалист по развитию и контролю сервиса,Газпром нефть,0.156349


# Раздел 2: Разработанные функции

In [47]:
# функция сбора вакансий
# принимает на вход список с идентификаторами компаний с сайта hh.ru
# одним из способов получить идентификатор - скопировать его из ссылки на любую вакансию интересующей компании 
def get_vacancies(employer_id_list):
    num_per_page = 100
    all_vacancy_ids = []

    # соберем идентификаторы вакансий по каждому работодателю
    for employer in employer_id_list:
        url = f'https://api.hh.ru/vacancies?employer_id={employer}&per_page={num_per_page}'
        res = requests.get(url)
        number_of_pages = res.json().get('pages')

        for i in tqdm(range(number_of_pages)):
            url = f'https://api.hh.ru/vacancies?employer_id={employer}&page={i}&per_page={num_per_page}'
            res = requests.get(url)
            vacancies = res.json()
            vacancy_ids = [el.get('id') for el in vacancies.get('items')]
            all_vacancy_ids.extend(vacancy_ids)
        
    # соберем данные по вакансиям

    # рассчитаем число шагов для сбора (для минимизации шансов отключения сессии)
    step = 200
    number_of_steps = math.ceil(len(all_vacancy_ids)/step)

    for stp in range(number_of_steps):
        all_vacancy_descriptions=pd.DataFrame(columns=['id', 'name', 'description', 'employer.name'])
        loop_ids = all_vacancy_ids[stp*step:(stp+1)*step]

        for i in tqdm(loop_ids):

            url = f'https://api.hh.ru/vacancies/{i}'
            res = requests.get(url)
            # нормализация json
            vacancies = pd.json_normalize(res.json(), max_level=1).filter(['id', 'name', 'description', 'employer.name'])

            all_vacancy_descriptions = pd.concat([all_vacancy_descriptions, vacancies], join="outer")

        # сохранение в csv
        all_vacancy_descriptions.to_csv(f'descriptions_{stp}.csv')
        # ожидание, чтобы минимизировать шансы на завершение сессии со стороны API hh.ru
        time.sleep(10)
    
    return None
        

# функция лемматизации (заимствованно с блока Data Science)
def lemmatize(row):
    morph = pymorphy2.MorphAnalyzer()
    t = []
    for word in row.split():
        if word in ['strong', 'ul', 'li']:
            t.append('')
        elif len(word)<=3:
            continue
        else:
            p = morph.parse(word)[0]
            t.append(p.normal_form)
    return " ".join(t)

# функция расчета и отображения ТОПа вакансий
# принимает на вход имя файла с описанием опыта кандидата 
# и число случайно выбираемых описаний вакансий
def get_top_vacancies(candidate_experience_file, number_of_random_rows):
    # загрузим сохраненные данные
    df = pd.DataFrame()

    for f in glob.glob('descriptions_*.csv'):
        df = pd.concat([df, pd.read_csv(f, index_col='id')], join="outer")

    df = df.drop(columns='Unnamed: 0')
    
    # выберем случайные N вакансий
    df = df.sample(number_of_random_rows, ignore_index=True)
    
    # скачаем стопслова
    try:
        _create_unverified_https_context = ssl._create_unverified_context
    except AttributeError:
        pass
    else:
        ssl._create_default_https_context = _create_unverified_https_context

    nltk.download('stopwords')

    stopwords_rus = stopwords.words('russian')

    # дополним лишними элементами (html, css)
    stopwords_rus.extend(['<strong>', '</strong>', 'strong', '<ul>', '</ul>', 'ul', '<li>', 
                          '</li>', 'li', '<p>', '</p>', 'p', 'n', 'u'])
    
    # загрузим описание опыта
    with open(candidate_experience_file, encoding='utf-8') as f:
        experience = pd.Series(f.read())
        
    
    # объединим датафрейм с описанями вакансий и строку с опытом кандидата
    description_with_experience = pd.concat([df.description, experience], join="outer")
    description_with_experience = description_with_experience.reset_index(drop=True)

    description_with_experience.iloc[[0, -1]].replace('\n', ' ').replace('\u2028',' ')
    
    # применим функцию лемматизации к датасету
    description_with_experience = description_with_experience.apply(lemmatize)
    
    # настроим TF-IDF
    text_transformer = TfidfVectorizer(stop_words=stopwords_rus, ngram_range=(1,1), lowercase=True, max_features=10000)
    text = text_transformer.fit_transform(description_with_experience)
    
    pd.set_option('display.max_colwidth', None)

    # рассчитаем матрицу косинусных расстояний
    cos_similarity_matrix = cosine_similarity(text)

    # возьмем только столбец с расстояниями для опыта кандидата
    cos_similarity_series = pd.Series(cos_similarity_matrix[-1])

    # отбросим последнюю строку с расстоянием = 1 
    cos_similarity_series = cos_similarity_series[:-1]

    cos_similarity_series.index = df.index

    # добавим полученные расстояния обратно в датасет
    df['cos_similarity'] = cos_similarity_series[:-1]

    # отсортируем по убыванию косинусного расстояния
    df = df.sort_values(by='cos_similarity', ascending=False)

    # вернем в результате 10 наиболее подходящих вакансий
    return df[['name', 'employer.name', 'description', 'cos_similarity']].head(10)
  

## Вызов разработанных функций

In [48]:
# определим работодателей для сбора вакансий
employers = {'name': ['sber', 'sbertech', 'rosatom', 'gazprom_neft'], 'id': [3529, 906557, 577743, 39305]}

# используем функцию сбора данных
# get_vacancies(employers['id'])

# используем функцию Топа вакансий
result = get_top_vacancies('Описание опыта.txt', 100)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ilyanaboyshchikov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Анализ результатов

In [49]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words=stopwords_rus) 
X = vectorizer.fit_transform(result.description) 

features = pd.DataFrame(X.A, columns=vectorizer.get_feature_names_out())
type(features)

s = features.sum()
features_sort = pd.DataFrame(features[s.sort_values(ascending=False).index[:15]])

features_sort

,опыт,данных,работы,разработки,br,разработка,знание,задачи,проекты,участие,веб,кода,python,навыки,результатам
0,7,5,4,1,0,2,0,3,2,0,0,0,1,0,2
1,5,3,3,6,3,1,0,1,0,3,0,1,0,1,0
2,6,4,7,4,1,1,1,2,1,1,0,4,2,0,1
3,4,1,2,1,10,4,4,1,1,2,0,0,0,1,0
4,6,2,1,3,0,2,0,0,1,0,0,0,1,1,0
5,5,6,10,2,0,2,7,3,1,0,0,1,0,0,1
6,2,2,2,3,0,3,1,1,2,1,6,3,4,4,2
7,2,2,2,3,0,3,1,1,2,1,6,3,4,4,2
8,3,8,4,0,0,0,3,1,2,1,0,0,0,0,2
9,1,5,1,0,7,0,1,2,1,3,0,0,0,0,0
